In [1]:
 ! pip install -U spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 6.5 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 6.7 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 9.3 MB/s eta 0:00:009.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 9.3 MB/s eta 0:00:0031m9.7 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.16.3
    Uninstalling pydantic_core-2.16.3:
      Successfully uninstalled pydantic_core-2.16.3


In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib64/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib64/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/mnt/Disk/MediscanPlus/Server/.venv/lib64/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/mnt/Disk/MediscanPlus/Server/.venv/lib64/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/mn

In [5]:
import json
f = open('drug_composition_annotations.json')
TRAIN_DATA = json.load(f)

In [6]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 43/43 [00:00<00:00, 522.19it/s]


In [7]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     36.33    3.81    3.33    4.44    0.04
 16     200         18.63    864.05  100.00  100.00  100.00    1.00
 36     400          0.83      0.66  100.00  100.00  100.00    1.00
 61     600          0.01      0.01  100.00  100.00  100.00    1.00
 92     800          0.00      0.00  100.00  100.00  100.00    1.00
131    1000          0.00      0.00  100.00  100.00  100.00    1.00
178    1200          0.00      0.00  100.00  100.00  100.00    1.00
239    1400          0.00      0.00  100.00  100.00  100.00    1.00
306    1600          0.00      0.00 

In [9]:
nlp_ner = spacy.load("/content/model-best")

In [10]:
doc = nlp_ner("30 capsules Cipla Rx Formotero Fumarate and Budesonide Powder For Inhalation IP foracort rotacaps 200")

In [11]:

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [12]:
!zip -r /content/model-best.zip /content/model-best/

  adding: content/model-best/ (stored 0%)
  adding: content/model-best/config.cfg (deflated 61%)
  adding: content/model-best/tok2vec/ (stored 0%)
  adding: content/model-best/tok2vec/model (deflated 8%)
  adding: content/model-best/tok2vec/cfg (stored 0%)
  adding: content/model-best/tokenizer (deflated 81%)
  adding: content/model-best/vocab/ (stored 0%)
  adding: content/model-best/vocab/strings.json (deflated 71%)
  adding: content/model-best/vocab/lookups.bin (stored 0%)
  adding: content/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/model-best/vocab/key2row (stored 0%)
  adding: content/model-best/vocab/vectors (deflated 45%)
  adding: content/model-best/meta.json (deflated 58%)
  adding: content/model-best/ner/ (stored 0%)
  adding: content/model-best/ner/model (deflated 7%)
  adding: content/model-best/ner/cfg (deflated 33%)
  adding: content/model-best/ner/moves (deflated 38%)
